# Layer Specific Gene GO

In [1]:
import joblib
import pandas as pd
import pathlib
from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.test_data.genes_NCBI_10090_ProteinCoding import GENEID2NT as GeneID2nt_mus
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

## Load marker gene lists

In [2]:
layer = 'IT-L6'
output_dir = pathlib.Path(layer)
output_dir.mkdir(exist_ok=True)

In [3]:
gene_meta = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
                        sep='\t', index_col='gene_id')

In [4]:
layer_markers = joblib.load(f'../region_hypo_genes.{layer}.obj')
layer_markers_names = {k: [gene_meta.at[g, 'gene_name'] for g in v] for k, v in layer_markers.items()}
gene_id_base_to_name = {g.split('.')[0]: gene_meta.loc[g, 'gene_name'] for v in layer_markers.values() for g in v}
for k, v in layer_markers.items():
    print(k, len(v))

SSp 13
ACA 20
PFC 27
SSs 14
MOp 15
MOs 22
AI 11


In [5]:
ensembl_id_to_ncbi = pd.read_csv(
    '/home/hanliu/ref/ncbi/gene2ensembl.mouse.tsv.gz', sep='\t',
    index_col=2)['GeneID'].to_dict()
ncbi_to_ensembl_id = {v:k for k, v in ensembl_id_to_ncbi.items()}

In [6]:
# convert ensembl id to entrez id, some missing id is usually low conf gene
layer_markers_ncbi = {}
for k, v in layer_markers.items():
    v = [ensembl_id_to_ncbi[i.split('.')[0]] for i in v if i.split('.')[0] in ensembl_id_to_ncbi]
    layer_markers_ncbi[k] = v
    print(k, len(v))

SSp 13
ACA 20
PFC 25
SSs 13
MOp 14
MOs 22
AI 11


## Setup GO

In [7]:
obo_fname = download_go_basic_obo()
fin_gene2go = download_ncbi_associations()
obodag = GODag("go-basic.obo")

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[10090])
# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated mouse genes".format(NS=nspc, N=len(id2gos)))

  EXISTS: go-basic.obo
  EXISTS: gene2go
go-basic.obo: fmt(1.2) rel(2020-01-01) 47,337 GO Terms
HMS:0:00:04.293774 367,364 annotations READ: gene2go 
1 taxids stored: 10090
BP 17,860 annotated mouse genes
CC 18,826 annotated mouse genes
MF 16,723 annotated mouse genes


In [8]:
goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 60% 16,821 of 28,212 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 64% 18,172 of 28,212 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 58% 16,337 of 28,212 population items found in association


In [9]:
# combine all the genes
genes = list(set(sum(layer_markers_ncbi.values(), [])))
len(genes)

71

In [10]:

goea_results_all = goeaobj.run_study(genes)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
goeaobj.wr_tsv(str(f"{layer}.GO.tsv"), goea_results_sig)
plot_results(layer + ".GO{NS}.png", goea_results_sig)

# add more gene info
sig_go_df = pd.read_csv(f"{layer}.GO.tsv", sep='\t', index_col=0)

sig_go_df['study_items'] = sig_go_df['study_items'].fillna('')
sig_go_df['gene_ids'] = sig_go_df['study_items'].apply(lambda i: ','.join(
    [ncbi_to_ensembl_id[int(g)] for g in i.split(',') if g != '']))
sig_go_df['gene_names'] = sig_go_df['gene_ids'].apply(lambda i: ','.join(
    [gene_id_base_to_name[g] for g in i.split(',') if g != '']))
sig_go_df.to_csv(f"{layer}.GO.tsv", sep='\t')


Run BP Gene Ontology Analysis: current study set of 71 IDs ...
 91%     61 of     67 study items found in association
 94%     67 of     71 study items found in population(28212)
Calculating 12,254 uncorrected p-values using fisher_scipy_stats
  12,254 GO terms are associated with 16,821 of 28,212 population items
     564 GO terms are associated with     61 of     71 study items
  METHOD fdr_bh:
       1 GO terms found significant (< 0.05=alpha) (  1 enriched +   0 purified): statsmodels fdr_bh
       6 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 71 IDs ...
100%     67 of     67 study items found in association
 94%     67 of     71 study items found in population(28212)
Calculating 1,724 uncorrected p-values using fisher_scipy_stats
   1,724 GO terms are associated with 18,172 of 28,212 population items
     126 GO terms are associated with     67 of 